# Question and answer LLM using the Vector Database data as context

Prerequites: 

1. This notebook assumes you've completed the previous notebook and have data loaded into your vector store. 

2. Azure openAI endpoint
    Confirm that you've deployed both an embedding model and a LLM. https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal


In [ ]:
%pip install weaviate-client
%pip install langchain
%pip install openai[datalib]
%pip install tiktoken
%pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv("credentials.env", override=True)


In [ ]:
from langchain.vectorstores import Weaviate
import weaviate

WEAVIATE_URL = os.environ["VECTOR_DB_WEAVIATE_URL"]
WEAVIATE_API_KEY = os.environ["WEAVIATE_API_KEY"]

#create client to inte ract with Weaviate
client = weaviate.Client(url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY))
#Print schemas in weaviate, you should see your index from the previous notebook named "arxivcs_index"
client.schema.get()

### Set embedding parameters and LLM paramters

We are not embedding documents into Weaviate but we still need the embedding model to convert our prompt into a vector and do a similarity search. Make sure you are using the same embedding model here as you did to write data to the vector database.  

In [ ]:
#embedding parameters, should be the same as what was used to embed documents into weaviate
import openai

openai.api_type = "azure"
openai.api_key = os.environ['openai_api_key']
openai.api_base = os.environ["AZURE_OPENAI_ENDPOINT"]
openai.api_version = os.environ["AZURE_OPENAI_API_VERSION"]

In [ ]:
#LLM Paramaters
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_base=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    deployment_name=os.environ["AZURE_OPENAI_LLM_DEPLOYMENT"],
    openai_api_key=os.environ['openai_api_key'],
    openai_api_type="azure",
)

In [ ]:
from langchain import PromptTemplate, LLMChain

# Prompt
prompt = PromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)

# Chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

#Converting our input question to an Embedding to use for search
response = openai.Embedding.create(
    input="What is Quantum mechanics?",
    engine="textembedding"
)
embeddings = response['data'][0]['embedding']

# Run
db = Weaviate(client=client, index_name="arxivcs_index", text_key="text")
docs = db.similarity_search_by_vector(embedding=embeddings)
result = llm_chain(docs)

# Output
result['text']